In [18]:
import os
import json
import pandas as pd

In [2]:
from pymodbus.client import ModbusTcpClient
from pymodbus.exceptions import ModbusException

In [3]:
import time

In [6]:
#!/bin/python

from pyModbusTCP.server import ModbusServer, DataBank
from time import sleep
from random import uniform

# Create an instance of ModbusServer
server = ModbusServer("127.0.0.1", 502, no_block=True)

try:
    print("Start server...")
    server.start()
    print("Server is online")
    state = [0]
    while True:
        DataBank.set_words(0, [int(uniform(0, 100))])
        if state != DataBank.get_words(1):
            state = DataBank.get_words(1)
            print("Value of Register 1 has changed to " +str(state))
        sleep(0.5)

except:
    print("Shutdown server ...")
    server.stop()
    print("Server is offline")


Start server...
Shutdown server ...
Server is offline


In [7]:
client = ModbusTcpClient('127.0.0.1',port=502)

In [8]:
if client.connect():
    print("Connected to Modbus server")
else:
    print("Failed to connect to Modbus server")
    client.close()
    

Connected to Modbus server


In [9]:
def read_input(address):
    result = client.read_holding_registers(address, 1)
    return result.registers[0]

In [10]:
def write_output(address, value):
    result = client.write_register(address, value)
    print(f"Successfully wrote {value} to address {address}")

In [21]:
def execute_commands_from_json(json_file):
    if os.path.exists(json_file):
        with open(json_file, 'r') as file:
            data = json.load(file)
        df = pd.DataFrame(data['actions'])
        for index, row in df.iterrows():
            set_x = int(row['setX']) if pd.notna(row['setX']) else None
            set_y = int(row['setY']) if pd.notna(row['setY']) else None
            vacuum = int(row['vacuum']) if pd.notna(row['vacuum']) else None
            if set_x is not None and set_y is not None:
                set_crane_position(set_x, set_y)
            if vacuum is not None:
                if vacuum == 1:
                    activate_vacuum()
                else:
                    deactivate_vacuum()
            if set_x is not None and set_y is not None:
                wait_for_crane_to_reach(set_x, set_y)
    else:
        print(f"File {json_file} not found.")

In [12]:
def activate_vacuum():
    write_output(3, 1)
    print("Vacuum activated")

In [13]:
def deactivate_vacuum():
    write_output(3, 0)
    print("Vacuum deactivated")

In [20]:
def set_crane_position(x, y):
    write_output(1, 55)
    write_output(2, 44)
    print(f"Crane moving to position X: {x}, Y: {y}")

In [15]:
def get_crane_position():
    crane_atX = read_input(15)
    crane_atY = read_input(16)
    print(f"Crane is at position X: {crane_atX}, Y: {crane_atY}")
    return crane_atX, crane_atY

In [16]:
def wait_for_crane_to_reach(x, y):
    posX, posY = get_crane_position()
    while posX != x or posY != y:
        time.sleep(0.1)  # Short delay to prevent excessive polling
        posX, posY = get_crane_position()
    print("Crane reached the target position")

In [19]:
if __name__ == "__main__":
    if client.connect():
        print("Connected to Modbus server")
        json_file = 'C:\\Users\\rahma\\handson-ml2\\crane_simulation\\crane_commands.json'
        execute_commands_from_json(json_file)
        client.close()
    else:
        print("Failed to connect to Modbus server")


Connected to Modbus server
Successfully wrote 0 to address 3
Vacuum deactivated
Successfully wrote 55 to address 1
Successfully wrote 200 to address 2
Crane moving to position X: 55, Y: 200
Crane is at position X: 500, Y: 150
Crane is at position X: 500, Y: 150
Crane is at position X: 485, Y: 165
Crane is at position X: 470, Y: 180
Crane is at position X: 455, Y: 195
Crane is at position X: 440, Y: 200
Crane is at position X: 440, Y: 200
Crane is at position X: 425, Y: 200
Crane is at position X: 410, Y: 200
Crane is at position X: 395, Y: 200
Crane is at position X: 380, Y: 200
Crane is at position X: 365, Y: 200
Crane is at position X: 350, Y: 200
Crane is at position X: 335, Y: 200
Crane is at position X: 320, Y: 200
Crane is at position X: 305, Y: 200
Crane is at position X: 290, Y: 200
Crane is at position X: 275, Y: 200
Crane is at position X: 260, Y: 200
Crane is at position X: 245, Y: 200
Crane is at position X: 245, Y: 200
Crane is at position X: 230, Y: 200
Crane is at positi